# 6-2. 네이버 뉴스 기사 수집
## 용어정리
### 1) Web Scraping (웹 스크랩핑)
컴퓨터 소프트웨어 기술로 웹 사이트들에서 원하는 정보를 추출하는 것<br>
국내에서는 흔히 크롤링이라는 용어로 많이 알려져 있다.<br>
하나의 페이지에서 데이터 수집

### 2) 웹 크롤러
인터넷에 있는 웹 사이트를 방문하여 자료를 수집하는 일을 하는 프로그램<br>
즉, 하나의 페이지에서 파생되는 하위 페이지까지 일괄적으로 방문해 수집하는 것을 의미.<br>
엄청난 분량의 웹 문서를 사람이 일일이 구별하여 모으는 일을 불가능에 가까우므로 웹 문서 검색에서는 이를 자동 수행<br>
링크를 따라다시면서 복수의 페이지에서 데이터 수집

<hr/>

# #01. 필요한 모듈 참조

In [1]:
import requests
from bs4 import BeautifulSoup

# #02. 수집 준비
## 1) 접속을 수행하기 위한 session 객체 생성
> 웹 페이지로부터 데이터를 수집할 경우 항상 가장 처음에 위치해야 하는 코드입니다.

In [2]:
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36"
session= requests.Session()
session.headers.update({'User-agent':user_agent, 'referer':None})

## 2) 접근할 페이지 주소 (네이버 뉴스 기사)

In [3]:
content_url = 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=103&oid=001&aid=0012233834'

# #03. 데이터 수집
지정한 URL에 접속하고 접속 결과를 체크한다.

In [4]:
# 데이터 가져오기
r = session.get(content_url)

# 결과 검사
if r.status_code != 200:
    print("[%d Error] %s" % (r.status_code, r.reason))
    quit()

# #04. 데이터 추출
URL에 접속해서 가져온 HTML 코드에서 분석을 원하는 부분을 추출한다.
## 1) 가져온 HTML 코드 확인
웹 페이지의 인코딩 형식을 확인하여 설정해야 한다.

In [5]:
r.encoding='euc-kr'
#r.text

## 2) 기사 본문 추출하기

In [6]:
# 웹 페이지의 소스코들르 사용, HTML 분석 객체로 생성
soup = BeautifulSoup(r.text, 'html.parser')

# css 선택자를 활용해 가져오기를 원하는 부분 지정
selector = soup.select('#articleBodyContents')

#가져온 내용이 없다면?
if not selector:
    print('크롤링 실패')
    quit()
    
print(type(selector))
#print(len(selector))
selector

<class 'bs4.element.ResultSet'>


[<div class="_article_body_contents" id="articleBodyContents">
 <!-- 본문 내용 -->
 <!-- TV플레이어 -->
 <!-- // TV플레이어 -->
 <script type="text/javascript">
 // flash 오류를 우회하기 위한 함수 추가
 function _flash_removeCallback() {}
 </script>
 <strong style="display:block;overflow:hidden;position:relative;margin:33px 20px 10px 3px;padding-left:11px;font-weight:bold;border-left: 2px solid #141414;">주차장·화장실 등 상춘객 편의시설 일체 제공 안 해</strong><br/><br/><span class="end_photo_org"><img alt="" src="https://imgnews.pstatic.net/image/001/2021/03/02/AKR20210302038051052_05_i_P4_20210302114248554.jpg?type=w647"><em class="img_desc">2020년 진해 벚꽃 명소 폐쇄<br/>[연합뉴스 자료사진]</em></img></span><br/><br/>(창원=연합뉴스) 이정훈 기자 = 코로나19가 발생 1년이 넘도록 수그러들지 않으면서 경남 창원시가 전국 최대 봄꽃축제 진해 군항제를 2년 연속으로 취소한다.<br/><br/>    창원시는 3월말 열 예정이던 제59회 진해 군항제를 취소한다고 2일 발표했다.<br/><br/>    창원시는 코로나19 하루 확진자 수가 여전히 전국적으로 300∼400명 이상씩 발생하고, 3∼4월까지는 백신 접종도 많지 않아 시민 지난해에 이어 군항제를 열지 않기로 했다고 설명했다.<br/><br/>    진해구 여론도 군항제 개최에 부정적이었다.<br/><br/>    창원시는 진해구민 667명을 대상으

## 3) 추출한 본문 영역에서 불필요한 부분 제거/치환
### 본문 영역에 대한 결과 얻기
html의 id 속성을 통해 가져온 원소 = 그 페이지 내 고유한 영역 <br>
select() 함수의 결과가 list여도 실제 원소는 하나만 존재 

=> 리스트에 대한 0번째 요소에 직접 접근해도 무관

In [7]:
item = selector[0]

#print(type(item))
print(item)

<div class="_article_body_contents" id="articleBodyContents">
<!-- 본문 내용 -->
<!-- TV플레이어 -->
<!-- // TV플레이어 -->
<script type="text/javascript">
// flash 오류를 우회하기 위한 함수 추가
function _flash_removeCallback() {}
</script>
<strong style="display:block;overflow:hidden;position:relative;margin:33px 20px 10px 3px;padding-left:11px;font-weight:bold;border-left: 2px solid #141414;">주차장·화장실 등 상춘객 편의시설 일체 제공 안 해</strong><br/><br/><span class="end_photo_org"><img alt="" src="https://imgnews.pstatic.net/image/001/2021/03/02/AKR20210302038051052_05_i_P4_20210302114248554.jpg?type=w647"><em class="img_desc">2020년 진해 벚꽃 명소 폐쇄<br/>[연합뉴스 자료사진]</em></img></span><br/><br/>(창원=연합뉴스) 이정훈 기자 = 코로나19가 발생 1년이 넘도록 수그러들지 않으면서 경남 창원시가 전국 최대 봄꽃축제 진해 군항제를 2년 연속으로 취소한다.<br/><br/>    창원시는 3월말 열 예정이던 제59회 진해 군항제를 취소한다고 2일 발표했다.<br/><br/>    창원시는 코로나19 하루 확진자 수가 여전히 전국적으로 300∼400명 이상씩 발생하고, 3∼4월까지는 백신 접종도 많지 않아 시민 지난해에 이어 군항제를 열지 않기로 했다고 설명했다.<br/><br/>    진해구 여론도 군항제 개최에 부정적이었다.<br/><br/>    창원시는 진해구민 667명을 대상으로 군항제 개최 

### 불필요한 태그들 제거하기

In [8]:
# <script> 태그 제거
for target in item.find_all('script'):
    target.extract()
    
# <a> 태그 제거
for target in item.find_all('a'):
    target.extract()
    
# <span> 태그 제거
for target in item.find_all('span'):
    target.extract()
    
# <div> 태그 제거
for target in item.find_all('div'):
    target.extract()
    
# <iframe> 태그 제거
# 기사에 따라 iframe태그가 있는 경우도 없는 경우도 있음 (주로 광고노출을 위해 사용)
# 존재하지 않는 태그를 삭제하는 것은 문제X -> 기사 내용에 상관 없이 수행
for target in item.find_all('iframe'):
    target.extract()
    
# <br> 태그 치환
# <br> 태그는 줄바꿈 문자를 의미하므로 replace_with 함수를 이용하여 \n으로 치환
for target in item.find_all('br'):
    target.replace_with('\n')
    
# 결과 출력
item

<div class="_article_body_contents" id="articleBodyContents">
<!-- 본문 내용 -->
<!-- TV플레이어 -->
<!-- // TV플레이어 -->

<strong style="display:block;overflow:hidden;position:relative;margin:33px 20px 10px 3px;padding-left:11px;font-weight:bold;border-left: 2px solid #141414;">주차장·화장실 등 상춘객 편의시설 일체 제공 안 해</strong>



(창원=연합뉴스) 이정훈 기자 = 코로나19가 발생 1년이 넘도록 수그러들지 않으면서 경남 창원시가 전국 최대 봄꽃축제 진해 군항제를 2년 연속으로 취소한다.

    창원시는 3월말 열 예정이던 제59회 진해 군항제를 취소한다고 2일 발표했다.

    창원시는 코로나19 하루 확진자 수가 여전히 전국적으로 300∼400명 이상씩 발생하고, 3∼4월까지는 백신 접종도 많지 않아 시민 지난해에 이어 군항제를 열지 않기로 했다고 설명했다.

    진해구 여론도 군항제 개최에 부정적이었다.

    창원시는 진해구민 667명을 대상으로 군항제 개최 여부를 설문 조사했다.

    설문에 응한 주민 85%가 군항제 개최에 부정적인 입장을 냈다. 



창원시는 군항제 취소를 알리면서 상춘객들이 벚꽃피는 시기에 진해구 방문을 자제해 달라고 요청했다.

    심재욱 창원시 문화체육관광국장은 "주요 벚꽃명소를 폐쇄한 지난해와 달리 벚꽃 구경을 강제로 막지는 않겠지만, 주차장과 화장실 등 관광객 편의시설을 일체 제공하지 않는다"고 밝혔다. 

    매년 수백만 명이 찾는 진해 군항제는 봄꽃 축제의 대명사다.

    1963년 1회 축제를 시작으로 지난해까지 한 번도 취소된 적이 없었다.

    그러나 지난해 초 국내에서 첫 코로나19 확진자가 나오고 이후 전국에서 확진자가 속출하자 창원시는 사상 처음으로 제58회 군항

## 4) 최종 텍스트 추출
### 텍스트만 추출
앞뒤로 불필요한 개행문자(\n)가 포함되어있다.

In [9]:
news_content_body = item.text
#news_content_body

### 앞 뒤 공백 제거

In [10]:
news_content = news_content_body.strip()
#news_content

In [11]:
### 추출 결과를 텍스트로 저장
with open('네이버뉴스.txt', 'w', encoding="utf-8") as f:
    f.write(news_content)